# Load Libraries

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os
os.chdir("/home/dalai/GNN_E")
print(os.getcwd())

from scripts.models import *
import pandas as pd
from scripts.utils import *
from math import ceil
import gc


import torch
import torch.optim as optim

print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.__version__)

/home/dalai/GNN_E


/home/dalai/.conda/envs/gat_gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
cuda
1.13.1+cu116


# Load df all Movies

In [2]:
# Load all movies with labels csv
df_all_movies = pd.read_csv("data/processed/all_movies_labelled_13_single.csv")

num_classes = 13 # based on df_movie choosen
batch_size = 32
print(batch_size)

32


In [3]:
############à
#JUST FOR PROVA

df_all_movies = df_all_movies[df_all_movies.movie.isin([0,3])]

# Split in Train, Validation, Test
### FUCNTION TO SPLIT NOT CORRECT

# Create dataset (i.e. graph list)

In [ ]:
df_train, df_val, df_test = split_train_val_test_horizontally(
    df_all_movies, 
    percentage_train=0.8, 
    percentage_val=0.0, #0 to not have nay val set
    path_pickle_delay="data/raw/labels/run_onsets.pkl",
    path_movie_title_mapping="data/raw/labels/category_mapping_movies.csv", 
    tr_len=1.3
)

dataset_train = DatasetEmo(df=df_train, node_feat="symmetricwindow")
dataset_val= DatasetEmo(df=df_val, node_feat="symmetricwindow")
dataset_test = DatasetEmo(df=df_test, node_feat="symmetricwindow")

# Extarct the list of graphs of each dataset
graphs_list_train = dataset_train.get_graphs_list()
graphs_list_val = dataset_val.get_graphs_list()
graphs_list_test = dataset_test.get_graphs_list()



Movie: AfterTheRain
  Start Time (TR)+4: 62
  Total Length (TR): 293
  Train End (TR): 296
  Validation End (TR): 354
  Movie End (TR): 355

Movie: Chatter
  Start Time (TR)+4: 61
  Total Length (TR): 240
  Train End (TR): 253
  Validation End (TR): 301
  Movie End (TR): 301
Movies in this df: [0 3]
Creating the graph 0 1 0/236
Creating the graph 0 1 1/236
Creating the graph 0 1 2/236
Creating the graph 0 1 3/236
Creating the graph 0 1 4/236
Creating the graph 0 1 5/236
Creating the graph 0 1 6/236
Creating the graph 0 1 7/236
Creating the graph 0 1 8/236
Creating the graph 0 1 9/236
Creating the graph 0 1 10/236
Creating the graph 0 1 11/236
Creating the graph 0 1 12/236
Creating the graph 0 1 13/236
Creating the graph 0 1 14/236
Creating the graph 0 1 15/236
Creating the graph 0 1 16/236
Creating the graph 0 1 17/236
Creating the graph 0 1 18/236
Creating the graph 0 1 19/236
Creating the graph 0 1 20/236
Creating the graph 0 1 21/236
Creating the graph 0 1 22/236
Creating the graph

# Define Hyperparameters VIB

In [6]:
from types import SimpleNamespace

# from VIB param_parser.py
args = SimpleNamespace(
    dataset_name="IMDB-BINARY",
    backbone="GCN",
    graph_type="prob",
    graph_metric_type="mlp",
    repar=True,
    num_layers=2,
    hidden_dim=16,
    folds=10,
    epochs=200,
    lr=0.001,
    lr_decay_factor=0.5,
    lr_decay_step_size=50,
    weight_decay=5e-5,
    batch_size=batch_size,
    test_batch_size=batch_size,
    beta=0.00001,
    IB_size=16,
    num_per=16,
    feature_denoise=False,
    top_k=10,
    epsilon=0.3,
    graph_skip_conn=0.0,
    graph_include_self=True
)


# Istantiate the Model

In [26]:

# following VIB main.py
import VIB.gsl as gsl


num_node_features = graphs_list_train[0].x.shape[1]
num_classes = 13
print("\nnum_node_features : %d, num_classes : %d"%(num_node_features, num_classes))


model = gsl.VIBGSL(args, 
               num_node_features, 
               num_classes)
print(model.__repr__())


num_node_features : 9, num_classes : 13
[ Graph Learner metric type: mlp, Graph Type: prob ]
VIBGSL


In [17]:
if 'adj' in graphs_list_test[0]:
    print("h")

graphs_list_test[0]

Data(x=[414, 9], edge_index=[2, 170982], edge_attr=[170982, 1], y=11)

# Train

In [28]:
from VIB.train_eval import *
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Clip gradients to prevent explosion

my_train_and_evaluate(
    train_graphs_list = graphs_list_train,
    test_graphs_list = graphs_list_test,
    model = model,
    epochs = args.epochs, 
    batch_size = args.batch_size, 
    test_batch_size = args.test_batch_size,
    lr = args.lr, 
    lr_decay_factor = args.lr_decay_factor, 
    lr_decay_step_size = args.lr_decay_step_size,
    weight_decay = args.weight_decay, 
    logger=None
)

Create Dataloader...
Dataloader Created
Epoch 1
	batch n 1
	batch n 11
	batch n 21
	batch n 31
	batch n 41
	batch n 51
	batch n 61
	batch n 71
	batch n 81
	batch n 91
	batch n 101
	batch n 111
	batch n 121
	batch n 131
	batch n 141
	batch n 151
	batch n 161
	batch n 171
	batch n 181
	batch n 191
	batch n 201
	batch n 211
	batch n 221
	batch n 231
	batch n 241
	batch n 251
	batch n 261
	batch n 271
	batch n 281
	batch n 291
	batch n 301
	batch n 311
	batch n 321
	batch n 331
	batch n 341
	batch n 351
	batch n 361
	batch n 371
	batch n 381
	batch n 391
	batch n 401
Problem test acc 
{'epoch': 1, 'train_loss': 2.9916014208423762, 'train_acc': 0.2947302969065755, 'test_loss': 8.524507541169683, 'test_acc': 2}
Epoch 2
	batch n 1
	batch n 11
	batch n 21
	batch n 31
	batch n 41
	batch n 51
	batch n 61
	batch n 71
	batch n 81
	batch n 91
	batch n 101
	batch n 111
	batch n 121
	batch n 131
	batch n 141
	batch n 151
	batch n 161
	batch n 171
	batch n 181
	batch n 191
	batch n 201
	batch n 211
	b

KeyboardInterrupt: 

In [ ]:
#del model
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 4         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   31744 B  |   31744 B  |    6440 GB |    6440 GB |
|       from large pool |       0 B  |       0 B  |    5523 GB |    5523 GB |
|       from small pool |   31744 B  |   31744 B  |     916 GB |     916 GB |
|---------------------------------------------------------------------------|
| Active memory         |   31744 B  |   31744 B  |    6440 GB |    6440 GB |
|       from large pool |       0 B  |       0 B  |    5523 GB |